# Model Serving with the Python CLI 

The Konduit Python CLI provides a quick way to build model serving configuration. You can do this with YAML files. 



```yaml
serving:
  http_port: 1337
  input_data_format: NUMPY
  output_data_format: NUMPY
  log_timings: True
  extra_start_args: -Xmx8g
steps:
  python_step:
    type: PYTHON
    python_path: .
    python_code_path: ./simple.py
    python_inputs:
      first: NDARRAY
    python_outputs:
      second: NDARRAY
client:
    port: 1337
```


```bash
konduit serve --config ../yaml/konduit.yaml
konduit predict-numpy --config ../yaml/konduit.yaml --numpy_data ../data/bert/input-0.npy
konduit stop-server --config ../yaml/konduit.yaml
```

In [ ]:
!konduit serve --config ../yaml/konduit.yaml

## python_step

Python steps can take any argument that can be passed to `PythonConfig`.  
Specify a Python step as follows: 

```yaml
steps: 
  python_step: 
    type: PYTHON
    python_code: simple.py 
```

- tensor_data_types_config
- model_config_type=
- python_code=None,
- python_code_path=None,
- python_inputs=None,
- python_outputs=None,
- extra_inputs=None,
- python_path=None,
- return_all_inputs=None,

Effectivelly, Python pipelines steps built for the CLI can only accept single inputs and outputs. The default name for inputs and outputs is `default`. This should be the key of your inputs to `python_step`. 

In [45]:
from konduit import PythonConfig 
PythonConfig()

In [46]:
from konduit import PythonStep

In [ ]:
PythonStep()

In [47]:
from konduit import ModelStep

In [ ]:
from PIL import Image 
import numpy as np 
from konduit.load import server_from_file, client_from_file

### Simple Python example

```yaml
serving:
  http_port: 1337
  input_data_format: NUMPY
  output_data_format: NUMPY
  log_timings: True
  extra_start_args: -Xmx8g
steps:
  python_step:
    type: PYTHON
    python_path: .
    python_code_path: ./simple.py
    python_inputs:
      first: NDARRAY
    python_outputs:
      second: NDARRAY
client:
    port: 1337
```

In [4]:
konduit_yaml_path = "../yaml/simple.yaml"
server = server_from_file(konduit_yaml_path)
client = client_from_file(konduit_yaml_path)
print(client.predict(np.array(33)))
server.stop()

Starting server..

Server has started successfully.
Starting server..

Server has started successfully.
[35]


In [ ]:
!konduit stop-server --config ../yaml/konduit.yaml

## PyTorch

```yaml
serving:
  http_port: 1337
  input_data_format: NUMPY
  output_data_format: NUMPY
  log_timings: True
  extra_start_args: -Xmx8g
steps:
  python_step:
    type: PYTHON
    python_path: .
    python_code_path: ../python/pytorch.py
    input_names: input1
    python_inputs:
      image: NDARRAY
    python_outputs:
      img_out_y: NDARRAY
client:
    port: 1337
```

In [44]:
im = Image.open("../data/facedetector/1.jpg")
im = np.array(im).astype("int")

konduit_yaml_path = "../yaml/pytorch.yaml"
server = server_from_file(konduit_yaml_path)
client = client_from_file(konduit_yaml_path)
print(client.predict({"default": im}))
server.stop()

TypeError: __init__() got an unexpected keyword argument 'input_names'

# ModelStep 
Unlike Python pipeline steps, ModelStep pipeline steps require a named input. 

## Deeplearning4j
```yaml
serving:
  http_port: 1337
  input_data_format: NUMPY
  output_data_format: NUMPY
  log_timings: True
  extra_start_args: -Xmx8g
steps:
  dl4j_mln_step:
    type: MULTI_LAYER_NETWORK
    model_loading_path: ../data/multilayernetwork/SimpleCNN.zip
    input_names: 
    - image_array
    output_names: 
    - output
    input_data_types:
      image_array: FLOAT
client:
    port: 1337
```

In [16]:
rand_image = np.random.randint(255, size=(3, 224, 224)) / 255
konduit_yaml_path = "../yaml/deeplearning4j.yaml"
server = server_from_file(konduit_yaml_path)
client = client_from_file(konduit_yaml_path)
print(client.predict({"image_array": rand_image}))
server.stop()

Starting server..

Server has started successfully.
[[4.1096203e-02 3.2491919e-01 2.5245989e-02 4.0063278e-05 6.0869849e-01]]


## Tensorflow 1.x

In [24]:
konduit_yaml_path = "../yaml/tensorflow-bert.yaml"
server = server_from_file(konduit_yaml_path)
client = client_from_file(konduit_yaml_path)
data_input = {
    'IteratorGetNext:0': np.load('../data/bert/input-0.npy'),
    'IteratorGetNext:1': np.load('../data/bert/input-1.npy'),
    'IteratorGetNext:4': np.load('../data/bert/input-4.npy')
}

print(client.predict(data_input))
server.stop()

Starting server.........................

The server wasn't able to start.


ERROR:root:HTTPConnectionPool(host='localhost', port=1337): Max retries exceeded with url: /healthcheck (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000021841F011C8>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))
Unable to connect to the server or the server health checks have failed. Please verify that the server is running without any issues...
ERROR:root:Unable to connect to the server at http://localhost:1337


NameError: name 'exit' is not defined

In [42]:
konduit_yaml_path = "../yaml/datavec.yaml"
server = server_from_file(konduit_yaml_path)
client = client_from_file(konduit_yaml_path)
print(client.predict({"first": "value"}))
server.stop()

KeyError: 'type'

In [33]:
import yaml 
with open(konduit_yaml_path, 'r') as f:
    data = yaml.safe_load(f)